In [1]:
import requests 
from bs4 import BeautifulSoup

In [2]:
url = "https://in.tradingview.com/symbols/NSE-PAVNAIND/"
resp = requests.get(url)
print(resp.status_code)
soup = None
if(resp.status_code == 200):
    soup = BeautifulSoup(resp.text,'html.parser')


200


In [9]:
soup_div = soup.find_all('div',{'class':"container-lu7Cy9jC"})[2]

In [11]:
soup_div

<div class="container-lu7Cy9jC"><div class="wrapper-Hy9Labru title-lu7Cy9jC"><div class="container-Hy9Labru"><h2 class="title-Hy9Labru title-small-Hy9Labru titleHeader-lu7Cy9jC">About PAVNA INDUSTRIES LTD</h2></div></div><div class="content-gdSWdaJr"><div class="container-GRoarMHL"><div class="block-GgmpMpKr"><div class="labelWrapper-GgmpMpKr"><div class="apply-overflow-tooltip label-GgmpMpKr">Sector</div></div><div class="wrapper-GgmpMpKr"><a class="link-GgmpMpKr" href="/markets/stocks-india/sectorandindustry-sector/producer-manufacturing/"><div class="apply-overflow-tooltip value-GgmpMpKr">Producer Manufacturing</div><span class="iconArrow-GgmpMpKr"><svg height="18" viewbox="0 0 18 18" width="18" xmlns="http://www.w3.org/2000/svg"><path d="M11.78 8.375a1 1 0 010 1.25l-4 5a1 1 0 11-1.56-1.25L9.72 9l-3.5-4.375a1 1 0 111.56-1.25l4 5z" fill="currentColor"></path></svg></span></a><div class="buttons-GgmpMpKr"></div></div></div><div class="block-GgmpMpKr"><div class="labelWrapper-GgmpMpKr"

In [21]:
info = {
    "Sector": soup_div.find("div", text="Sector").find_next_sibling("div").text.strip(),
    "Industry": soup_div.find("div", text="Industry").find_next_sibling("div").text.strip(),
    "CEO": soup_div.find("div", text="CEO").find_next_sibling("div").text.strip(),
    "Website": soup_div.find("div", text="Website").find_next_sibling("div").find("a").text.strip(),
    "Headquarters": soup_div.find("div", text="Headquarters").find_next_sibling("div").text.strip(),
    "Employees (FY)": soup_div.find("div", text="Employees (FY)").find_next_sibling("div").text.strip(),
    "Founded": soup_div.find("div", text="Founded").find_next_sibling("div").text.strip(),
    "About-us": soup_div.find("div", class_="container-OkHxJmnJ").find("span").text.strip()
}
info

{'Sector': 'Producer Manufacturing',
 'Industry': 'Auto Parts: OEM',
 'CEO': 'Ravindra J. Pise',
 'Website': 'pavnagroup.com',
 'Headquarters': 'Aligarh',
 'Employees (FY)': '1.3K',
 'Founded': '1971',
 'About-us': 'Pavna Industries Ltd. engages in the manufacture and sale of automotive parts. The company offers various automobile parts, which includes ignition switches, fuel tank caps, latches, auto locks, handles, switches, oil pump, carburetor, throttle body, fuel cocks, injection system, casting components and other automotive parts solutions for automobile applications, serving automobile, agricultural and other applications across the world. The company was founded by Pawan Jain on April 19, 1994 and is headquartered in Aligarh, India.'}

In [46]:
import re

def convert_employees_format(employees_str):
    # Remove non-numeric characters
    numeric_str = re.sub(r'[^0-9.]', '', employees_str)

    # Convert K to thousands and M to millions
    if 'K' in employees_str:
        return int(float(numeric_str) * 1000)
    elif 'M' in employees_str:
        return int(float(numeric_str) * 1000000)
    else:
        return int(numeric_str)
    
def getTradingViewInfo(nse_symbol,ex='NSE',n_div=2):
    nse_symbol = nse_symbol.replace("-","_")
    url = f"https://in.tradingview.com/symbols/{ex}-{nse_symbol}/"
    resp = requests.get(url)
    print(resp.status_code,end=" ")
    if(resp.status_code == 404):
        print('\nretry with bse',end=" ")
        ex="BSE"
        url = f"https://in.tradingview.com/symbols/{ex}-{nse_symbol}/"
        resp = requests.get(url)
        print(resp.status_code,end=" ")
    soup = None
    if(resp.status_code == 200):
        soup = BeautifulSoup(resp.text,'html.parser')
        soup_div = soup.find_all('div',{'class':"container-lu7Cy9jC"})[n_div]
        sector_div = soup_div.find("div", text="Sector")
        industry_div = soup_div.find("div", text="Industry")
        ceo_div = soup_div.find("div", text="CEO")
        website_div = soup_div.find("div", text="Website")
        hq_div = soup_div.find("div", text="Headquarters")
        employees_div = soup_div.find("div", text="Employees (FY)")
        founded_div = soup_div.find("div", text="Founded")
        about_us_div = soup_div.find("div", class_="container-OkHxJmnJ")
        
        info = {}
        
        if(soup_div):
            if(sector_div):
                 info["Sector"]= sector_div.find_next_sibling("div").text.strip()
            if(industry_div):
                 info["Industry"]= industry_div.find_next_sibling("div").text.strip()
            if(ceo_div):
                info["CEO"]=ceo_div.find_next_sibling("div").text.strip()
            if(website_div):
                info["Website"]=website_div.find_next_sibling("div").text.strip()
            if(hq_div):
                info["Headquarters"]=hq_div.find_next_sibling("div").text.strip()
            if(employees_div):
                info["Headquarters"]=convert_employees_format(employees_div.find_next_sibling("div").text.strip())
                
            if(founded_div):
                info["Founded"]=founded_div.find_next_sibling("div").text.strip()
            if(about_us_div):
                info["About-us"]=about_us_div.find("span").text.strip()
        
#         info = {
#             "Sector": soup_div.find("div", text="Sector").find_next_sibling("div").text.strip(),
#             "Industry": soup_div.find("div", text="Industry").find_next_sibling("div").text.strip(),
#             "CEO": soup_div.find("div", text="CEO").find_next_sibling("div").text.strip(),
#             "Website": soup_div.find("div", text="Website").find_next_sibling("div").find("a").text.strip(),
#             "Headquarters": soup_div.find("div", text="Headquarters").find_next_sibling("div").text.strip(),
#             "Employees (FY)": convert_employees_format(soup_div.find("div", text="Employees (FY)").find_next_sibling("div").text.strip()),
#             "Founded": soup_div.find("div", text="Founded").find_next_sibling("div").text.strip(),
#             "About-us": soup_div.find("div", class_="container-OkHxJmnJ").find("span").text.strip()
#         }
        return info
    return

In [67]:
url = "https://www.moneycontrol.com/india/stockpricequote/auto-ancillaries/subros/S07"
resp = requests.get(url)
print(resp.status_code)
soup = None
if(resp.status_code == 200):
    soup = BeautifulSoup(resp.text,'html.parser')
    
ul_tag = soup.find('ul', class_='comp_inf company_slider')

# Find the <span> tag with text "NSE:"
span_nse_tag = ul_tag.find('span', string='NSE:')

# Extract the NSE symbol from the adjacent <p> tag
nse_symbol = span_nse_tag.find_next('p').text.strip()

print("NSE Symbol:", nse_symbol)

200
NSE Symbol: SUBROS


In [4]:
def get_nse_symbol(link):
    resp = requests.get(link)
    print(resp.status_code,end=" ")
    soup = None
    if(resp.status_code == 200):
        soup = BeautifulSoup(resp.text,'html.parser')

        ul_tag = soup.find('ul', class_='comp_inf company_slider')

        # Find the <span> tag with text "NSE:"
        span_nse_tag = ul_tag.find('span', string='NSE:')

        # Extract the NSE symbol from the adjacent <p> tag
        nse_symbol = span_nse_tag.find_next('p').text.strip()

        print("NSE: ", nse_symbol)
        return nse_symbol
    return 

In [75]:
get_nse_symbol(url)

200 NSE:  SUBROS


'SUBROS'

In [28]:
convert_employees_format("13K")

13000

In [25]:
getTradingViewInfo("SUBROS")

200


{'Sector': 'Producer Manufacturing',
 'Industry': 'Industrial Machinery',
 'CEO': 'Pramod Kumar Duggal',
 'Website': 'subros.com',
 'Headquarters': 'New Delhi',
 'Employees (FY)': 4633,
 'Founded': '1985',
 'About-us': 'Subros Ltd. engages in the manufacture of thermal products for automotive applications. Its products include compressors, condensers, heating, ventilation, and air conditioning systems, tubes and hoses, bus air conditioners, rail air conditioning, transport refrigeration system, truck cabin air conditioning system, and off-roader air conditioning. The company was founded by Ramesh Suri on February 14, 1985 and is headquartered in New Delhi, India.'}

In [6]:
import pandas as pd

about_us_old_data = pd.read_csv('./about-us-with-nse-symbol.csv')
about_us_old_data.head()

,Name,Sector,Link,Website,About us,NSE symbol
0,Carborundum Universal Ltd.,Abrasives,https://www.moneycontrol.com/india/stockpriceq...,http://www.cumi-murugappa.com,"Carborundum Universal Ltd,popularly known as C...",CARBORUNIV
1,Grindwell Norton Ltd.,Abrasives,https://www.moneycontrol.com/india/stockpriceq...,http://www.grindwellnorton.co.in,Grindwell Norton Ltd is Indias leading manufac...,GRINDWELL
2,Wendt (India) Ltd.,Abrasives,https://www.moneycontrol.com/india/stockpriceq...,http://www.wendtindia.com,Wendt (India) Ltd is a leading producer of Dia...,WENDT
3,Apollo Micro Systems Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,https://apollo-micro.com,NaN,APOLLO
4,Bharat Electronics Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.bel-india.com,Bharat Electronics Limited (BEL) was set up at...,BEL


In [26]:
get_nse_symbol(about_us_old_data.iloc[396]['Link'])

200 NSE:  SALZERELEC


'SALZERELEC'

In [25]:
about_us_old_data.iloc[396]['Link']

'https://www.moneycontrol.com/india/stockpricequote/electric-equipment/salzerelectronics/SE01'

In [31]:
# symbols = {}
for index,row in about_us_old_data[4500:].iterrows():
    print(index,end=" ") 
    url = row['Link']
    if url:
        symbol = get_nse_symbol(url)
        row['NSE symbol'] = symbol
        symbols[index] = symbol
        

4500 200 NSE:  
4501 200 NSE:  
4502 200 NSE:  
4503 200 NSE:  
4504 200 NSE:  
4505 200 NSE:  
4506 200 NSE:  KHAITANLTD
4507 200 NSE:  
4508 200 NSE:  
4509 200 NSE:  KOTHARIPRO
4510 200 NSE:  
4511 200 NSE:  
4512 200 NSE:  LANDMARK
4513 200 NSE:  
4514 200 NSE:  
4515 200 NSE:  
4516 200 NSE:  
4517 200 NSE:  
4518 200 NSE:  MADHURFOOD
4519 200 NSE:  
4520 200 NSE:  
4521 200 NSE:  540401
4522 200 NSE:  
4523 200 NSE:  
4524 200 NSE:  
4525 200 NSE:  
4526 200 NSE:  MMTC
4527 200 NSE:  
4528 200 NSE:  
4529 200 NSE:  
4530 200 NSE:  
4531 200 NSE:  MSTCLTD
4532 200 NSE:  
4533 200 NSE:  
4534 200 NSE:  
4535 200 NSE:  
4536 200 NSE:  
4537 200 NSE:  
4538 200 NSE:  
4539 200 NSE:  
4540 200 NSE:  
4541 200 NSE:  
4542 200 NSE:  
4543 200 NSE:  
4544 200 NSE:  
4545 200 NSE:  
4546 200 NSE:  
4547 200 NSE:  
4548 200 NSE:  
4549 200 NSE:  
4550 200 NSE:  
4551 200 NSE:  PRIYADYES
4552 200 NSE:  PTL
4553 200 NSE:  
4554 200 NSE:  
4555 200 NSE:  RACE
4556 200 NSE:  
4557 200 NSE:  RA

In [29]:
len(about_us_old_data)

4714

In [32]:
len(symbols.keys())


4714

In [33]:
import json

fp = open('symbols.json','w')
json.dump(symbols,fp)
fp.close()

In [34]:
about_us_old_data['NSE symbol'] = symbols

In [36]:
about_us_old_data.to_csv("about-us-with-nse-symbol.csv",index=False)

In [7]:
about_us_old_data[about_us_old_data['About us'].isnull() & about_us_old_data['NSE symbol'].isnull()]
# about_us_old_data['About us'].isnull()

,Name,Sector,Link,Website,About us,NSE symbol
10,TAAL Enterprises Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.taalent.co.in,NaN,NaN
15,Camson Seeds Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,https://www.camsonseedslimited.in,NaN,NaN
23,Indo US Bio-Tech Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,http://www.indousagriseeds.com,NaN,NaN
35,Satyam Silk Mills RE Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,NaN
39,Vrundavan Plantation Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,http://www.vrundavanplantation.com,NaN,NaN
...,...,...,...,...,...,...
4636,Hazoor Multi Projects RE Ltd.,TV Broadcasting & Software Production,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,NaN
4640,Maagh Advertising and Marketing Services Ltd.,TV Broadcasting & Software Production,https://www.moneycontrol.com/india/stockpriceq...,http://www.maaghadvertising.in,NaN,NaN
4649,Sadhna Broadcast Ltd Ltd.,TV Broadcasting & Software Production,https://www.moneycontrol.com/india/stockpriceq...,http://www.sadhna.com,NaN,NaN
4684,Poona Dal and Oil Industries Ltd.,Vegetable Oils & Products,https://www.moneycontrol.com/india/stockpriceq...,https://www.pdoil.co.in,NaN,NaN


In [8]:
# about_us_old_data[ about_us_old_data['NSE symbol']=='']
about_us_old_data['NSE symbol'].fillna("",inplace=True)

In [9]:
about_us_old_data[about_us_old_data['NSE symbol'] == '']

,Name,Sector,Link,Website,About us,NSE symbol
9,Sika Interplant Systems Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.sikaglobal.com,Sika Interplant Systems Ltd is an engineering ...,
10,TAAL Enterprises Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.taalent.co.in,NaN,
13,Beeyu Overseas Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,http://www.beeyuoverseas.in,Beeyu Overseas Ltd is engaged in the of manufa...,
15,Camson Seeds Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,https://www.camsonseedslimited.in,NaN,
17,Diana Tea Company Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,http://www.dianatea.in,Diana Tea Company Ltd is a tea-producing compa...,
...,...,...,...,...,...,...
4704,Duroply Industries Ltd.,Wood & Wood Products,https://www.moneycontrol.com/india/stockpriceq...,http://www.duroply.in,Duroply Industries Limited is engaged in plywo...,
4709,Milestone Furniture Ltd.,Wood & Wood Products,https://www.moneycontrol.com/india/stockpriceq...,http://www.milestonefurniture.in,NaN,
4710,Oriental Rail Infrastructure Ltd.,Wood & Wood Products,https://www.moneycontrol.com/india/stockpriceq...,http://www.orientalrail.com,Oriental Veneer Products Ltd is engaged in the...,
4711,Pratik Panels Ltd.,Wood & Wood Products,https://www.moneycontrol.com/india/stockpriceq...,http://www.pratikpanels.com,"Pratik Panels Ltd has Veneer Mill, Sawmill, Pl...",


In [10]:
no_data_index = about_us_old_data[about_us_old_data['About us'].isnull() & (about_us_old_data['NSE symbol'] == '')].index
about_us_old_data.iloc[no_data_index]

,Name,Sector,Link,Website,About us,NSE symbol
10,TAAL Enterprises Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.taalent.co.in,NaN,
15,Camson Seeds Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,https://www.camsonseedslimited.in,NaN,
23,Indo US Bio-Tech Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,http://www.indousagriseeds.com,NaN,
35,Satyam Silk Mills RE Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,
39,Vrundavan Plantation Ltd.,Agriculture,https://www.moneycontrol.com/india/stockpriceq...,http://www.vrundavanplantation.com,NaN,
...,...,...,...,...,...,...
4636,Hazoor Multi Projects RE Ltd.,TV Broadcasting & Software Production,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,
4640,Maagh Advertising and Marketing Services Ltd.,TV Broadcasting & Software Production,https://www.moneycontrol.com/india/stockpriceq...,http://www.maaghadvertising.in,NaN,
4649,Sadhna Broadcast Ltd Ltd.,TV Broadcasting & Software Production,https://www.moneycontrol.com/india/stockpriceq...,http://www.sadhna.com,NaN,
4684,Poona Dal and Oil Industries Ltd.,Vegetable Oils & Products,https://www.moneycontrol.com/india/stockpriceq...,https://www.pdoil.co.in,NaN,


In [11]:
about_us_old_data.drop(index=no_data_index,inplace=True)

In [12]:
len(about_us_old_data)

4177

In [13]:
about_us_old_data[about_us_old_data['About us'].isnull()]

,Name,Sector,Link,Website,About us,NSE symbol
3,Apollo Micro Systems Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,https://apollo-micro.com,NaN,APOLLO
5,Data Patterns (India) Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.datapatternsindia.com,NaN,DATAPATTNS
6,Dreamfolks Services Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.dreamfolks.in,NaN,DREAMFOLKS
8,MTAR Technologies Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.mtar.in,NaN,MTARTECH
59,Hardwyn India Ltd.,Aluminium & Aluminium Products,https://www.moneycontrol.com/india/stockpriceq...,http://www.hardwyn.com,NaN,HARDWYN
...,...,...,...,...,...,...
4673,M K Proteins Ltd.,Vegetable Oils & Products,https://www.moneycontrol.com/india/stockpriceq...,http://www.mkproteins.in,NaN,MKPL
4675,Manorama Industries Ltd.,Vegetable Oils & Products,https://www.moneycontrol.com/india/stockpriceq...,http://www.manoramagroup.co.in,NaN,MANORAMA
4697,Ethos Ltd.,Watches & Accessories,https://www.moneycontrol.com/india/stockpriceq...,http://www.ethoswatches.com,NaN,ETHOSLTD
4700,Archidply Decor Ltd.,Wood & Wood Products,https://www.moneycontrol.com/india/stockpriceq...,http://www.archidplydecor.com,NaN,ADL


In [14]:
about_us_old_data[about_us_old_data['About us'].isnull()][:6]

,Name,Sector,Link,Website,About us,NSE symbol
3,Apollo Micro Systems Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,https://apollo-micro.com,NaN,APOLLO
5,Data Patterns (India) Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.datapatternsindia.com,NaN,DATAPATTNS
6,Dreamfolks Services Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.dreamfolks.in,NaN,DREAMFOLKS
8,MTAR Technologies Ltd.,Aerospace & Defence,https://www.moneycontrol.com/india/stockpriceq...,http://www.mtar.in,NaN,MTARTECH
59,Hardwyn India Ltd.,Aluminium & Aluminium Products,https://www.moneycontrol.com/india/stockpriceq...,http://www.hardwyn.com,NaN,HARDWYN
76,Apex Frozen Foods Ltd.,Aquaculture,https://www.moneycontrol.com/india/stockpriceq...,http://www.apexfrozenfoods.in,NaN,APEX


In [153]:
# trading_view_data = {}
for index,row in about_us_old_data[about_us_old_data['About us'].isnull()].iloc[250:].iterrows():
#     if(not trading_view_data[index]):
    print(index,row['NSE symbol'],end=" ")
    trading_view_data[index] = getTradingViewInfo(row['NSE symbol'])
    print()

1832 SGBJUL28IV 404 
retry with bse 404 
1833 SGBJUL29IV 404 
retry with bse 404 
1834 SGBJUN29II 404 
retry with bse 404 
1835 SGBJUN30 404 
retry with bse 404 
1836 SGBJUN31I 404 
retry with bse 404 
1837 SGBMAR24 404 
retry with bse 404 
1838 SGBMAR25 404 
retry with bse 404 
1839 SGBMAR30X 404 
retry with bse 404 
1840 SGBMAY26 404 
retry with bse 200 
1841 SGBMAY28 404 
retry with bse 200 
1842 SGBMAY29I 404 
retry with bse 404 
1843 SGBMR29XII 404 
retry with bse 404 
1844 SGBN28VIII 404 
retry with bse 404 
1845 SGBNOV23 404 
retry with bse 404 
1846 SGBNOV24 404 
retry with bse 404 
1847 SGBNV29VII 404 
retry with bse 404 
1848 SGBSEP24 404 
retry with bse 404 
1849 SGBSEP28VI 404 
retry with bse 404 
1850 SGBSEP29VI 404 
retry with bse 404 
1859 SMCGLOBAL 200 
1883 UTIAMC 200 
1905 AHL 200 
1934 CKFSL 404 
retry with bse 200 
1947 CREDITACC 200 
1959 FEDFINA 200 
1962 FIVESTAR 200 
1965 FUSION 200 
2002 IRFC 200 
2023 M&MFIN-RE 404 
retry with bse 404 
2106 SPANDANA 200 
2142 

In [154]:
len(trading_view_data.keys())

585

In [156]:
fp = open("trading-view-few-points.json",'w')
json.dump(trading_view_data,fp)
fp.close()

In [148]:
getTradingViewInfo(about_us_old_data.loc[1051]['NSE symbol'])

404 
retry with bse 404 

In [152]:
about_us_old_data.loc[1430]['Link']

'https://www.moneycontrol.com/india/stockpricequote/miscellaneous/bharatbondetf-april2023/EBBET16222'

In [17]:
import json
fp = open('./trading-view-few-points.json','r')
trading_view_data = json.load(fp)
trading_view_data

{'3': {'Sector': 'Electronic Technology',
  'Industry': 'Electronic Production Equipment',
  'CEO': 'Karunakar Reddy Baddam',
  'Website': 'apollo-micro.com',
  'Headquarters': 298,
  'Founded': '1997',
  'About-us': 'Apollo Micro Systems Ltd. engages in the manufacture of electronic components and systems. It includes electronic manufacturing, hardware design, information technology and software, electronic, and mechanical services. It also offers space, and transportation solutions. The company was founded by Baddam Karunakar Reddy on March 3, 1997 and is headquartered in Hyderabad, India.'},
 '5': {'Sector': 'Electronic Technology',
  'Industry': 'Electronic Production Equipment',
  'CEO': 'Srinivasagopalan Rangarajan',
  'Website': 'datapatternsindia.com',
  'Headquarters': 947,
  'Founded': '1998',
  'About-us': 'Data Patterns (India) Ltd. engages in the manufacture of electronic boards and systems. Its products include radar and radar processor, avionics and electronic warfare, a

In [19]:
for index, row in about_us_old_data[about_us_old_data['About us'].isnull()].iterrows():
    a = trading_view_data[f'{index}']
    if a:
        about_us_old_data.loc[index, 'About us'] = a['About-us']

In [21]:
about_us_old_data.loc[3]

Name                                  Apollo Micro Systems Ltd.
Sector                                      Aerospace & Defence
Link          https://www.moneycontrol.com/india/stockpriceq...
Website                                https://apollo-micro.com
About us      Apollo Micro Systems Ltd. engages in the manuf...
NSE symbol                                               APOLLO
Name: 3, dtype: object

In [51]:

# about_us_old_data[about_us_old_data['About us'].isnull()]
for index,row in about_us_old_data[about_us_old_data['About us'].isnull()][:10].iterrows():
#     if(not trading_view_data[index]):
    print(index,row['NSE symbol'],end=" ")
    data = getTradingViewInfo(row['NSE symbol'],n_div=1)
    trading_view_data[index] = data
    if(data):
        about_us_old_data.loc[index, 'About us'] = data['About-us']

343 PLADAINFO 200 1051 RELIANCEP1 404 
retry with bse 404 1053 SAAKSHI 200 1430 EBBETF0423 404 
retry with bse 404 1433 CRMFGETF 200 1442 NIFTYEES 200 1490 RELNIFTY 404 
retry with bse 404 1741 L&TFH-RE 404 
retry with bse 404 1811 SGB202108 404 
retry with bse 404 1813 SGB212205 404 
retry with bse 404 

In [52]:
about_us_old_data[about_us_old_data['About us'].isnull()]
# soup = getTradingViewInfo(about_us_old_data.loc[1053]['NSE symbol'],n_div=1)

,Name,Sector,Link,Website,About us,NSE symbol
1051,Reliance Industries Partly Paid Up,Diversified,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,RELIANCEP1
1430,Bharat Bond ETF - April 2023,ETF,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,EBBETF0423
1433,Canara Robeco Gold Exchange Traded Fund Ltd.,ETF,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,CRMFGETF
1442,Edelweiss Nifty Exchange Traded Scheme Ltd.,ETF,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,NIFTYEES
1490,Reliance ETF Nifty BeEs,ETF,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,RELNIFTY
1741,L&T Finance Holdings - RE Ltd.,Finance - Investment,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,L&TFH-RE
1811,Sovereign Gold Bonds Scheme 2020-21 - Series V...,Finance - Investment,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,SGB202108
1813,Sovereign Gold Bonds Scheme 21-22 - Sr. V (Tr....,Finance - Investment,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,SGB212205
1817,SOVEREIGN GOLD BONDS 2.50% APRIL 2028 SR-I 202...,Finance - Investment,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,SGBAPR28I
1818,SOVEREIGN GOLD BONDS 2.75% AUG 2024 TR-IV Ltd.,Finance - Investment,https://www.moneycontrol.com/india/stockpriceq...,NaN,NaN,SGBAUG24


In [53]:
about_us_old_data.to_csv("max-comp-about-us.csv")